In [2]:
from pyspark.sql import SparkSession

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext("local", "test")

In [4]:
spark = SparkSession.builder.appName('new').getOrCreate()

In [5]:
DimeEmployee = spark.read.option('header','true').csv("C:/Users/aditya.singh490/Documents/AdventureWorks/AdventureWorks_DF/DimEmployee.csv")

In [6]:
DimeProduct = spark.read.option('header','true').csv("C:/Users/aditya.singh490/Documents/AdventureWorks/AdventureWorks_DF/DimProduct.csv")

In [7]:
FactResellerSale = spark.read.option('header','true').csv("C:/Users/aditya.singh490/Documents/AdventureWorks/AdventureWorks_DF/FactResellerSales.csv")

In [8]:
DimeEmployee_RDD = sc.textFile("C:/Users/aditya.singh490/Documents/AdventureWorks/AdventureWorks_RDD/DimEmployee.csv")

In [9]:
DimeProduct_RDD = sc.textFile("C:/Users/aditya.singh490/Documents/AdventureWorks/AdventureWorks_RDD/DimProduct.csv")

In [10]:
FactResellerSale_RDD = sc.textFile("C:/Users/aditya.singh490/Documents/AdventureWorks/AdventureWorks_RDD/FactResellerSales.csv")

In [13]:
def func(a):
    a.split(",")
DimeEmployee_Split = DimeEmployee_RDD.map(func)

In [23]:
data_list = ["ProductKey", "ProductSubcategoryKey", "EnglishProductName", "StandardCost", "Color", "ListPrice", "DealerPrice"]
DimeProduct_DF = DimeProduct.toDF(*data_list)

In [24]:
DimeProduct_DF.show()

+----------+---------------------+--------------------+------------+------------+---------+-----------+
|ProductKey|ProductSubcategoryKey|  EnglishProductName|StandardCost|       Color|ListPrice|DealerPrice|
+----------+---------------------+--------------------+------------+------------+---------+-----------+
|       233|                   21|LongSleeve Logo J...|     29.0807|       Multi|  48.0673|    28.8404|
|       196|                 null|      LL SpindleAxle|        null|          NA|     null|       null|
|        65|                 null|           Hex Nut 1|        null|          NA|     null|       null|
|       347|                    1|Mountain100 Silve...|   1912.1544|      Silver|  3399.99|   2039.994|
|       137|                 null|          Lock Nut 4|        null|          NA|     null|       null|
|       155|                 null|         Metal Angle|        null|          NA|     null|       null|
|       211|                   14|HL Road Frame  Re...|        n

In [37]:
DimeEmployee.show()

+-----------+-----------------+-----------------+---------+
|EmployeeKey|ParentEmployeeKey|SalesTerritoryKey|FirstName|
+-----------+-----------------+-----------------+---------+
|         72|               32|               11|     Ajay|
|         36|               87|               11|  Hemanth|
|        104|               16|               11|   Ganesh|
|        120|               44|               11|  Perumal|
|        287|              272|                4|      Shu|
|        105|               44|               11|      Dan|
|        201|               23|               11|     Lori|
|        130|               92|               11|     Sean|
|        295|              290|                8|   Rachel|
|        216|               92|               11|    Chris|
|        285|              272|                5|     Tsvi|
|         59|               18|               11|    Annik|
|         69|               16|               11|      Bob|
|         83|              214|         

In [40]:
FactResellerSale.show()

+----------+------------+----------+-----------+-----------+-----------+------------+-----------------+-------------+---------+--------------------+--------------+-------------------+----------------+-----------+--------+
|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|ResellerKey|EmployeeKey|PromotionKey|SalesTerritoryKey|OrderQuantity|UnitPrice|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount| Freight|
+----------+------------+----------+-----------+-----------+-----------+------------+-----------------+-------------+---------+--------------------+--------------+-------------------+----------------+-----------+--------+
|       339|    20070401|  20070413|   20070408|        523|        282|           1|                3|            2|  469.794|                   0|             0|           486.7066|        973.4132|    939.588| 23.4897|
|       213|    20060801|  20060813|   20060808|        403|        282|           1|                4|         

In [51]:
join_DF = DimeEmployee.join(other=FactResellerSale,on='EmployeeKey',how='inner')

In [52]:
join_DF.show()

+-----------+-----------------+-----------------+---------+----------+------------+----------+-----------+-----------+------------+-----------------+-------------+---------+--------------------+--------------+-------------------+----------------+-----------+--------+
|EmployeeKey|ParentEmployeeKey|SalesTerritoryKey|FirstName|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|ResellerKey|PromotionKey|SalesTerritoryKey|OrderQuantity|UnitPrice|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount| Freight|
+-----------+-----------------+-----------------+---------+----------+------------+----------+-----------+-----------+------------+-----------------+-------------+---------+--------------------+--------------+-------------------+----------------+-----------+--------+
|        282|              272|                4|    Linda|       339|    20070401|  20070413|   20070408|        523|           1|                3|            2|  469.794|                   0|  

In [57]:
join_DF.createOrReplaceTempView('EmpProduct')

In [59]:
EmpProduct_DF = spark.sql("select EmployeeKey, FirstName, TotalProductCost, SalesAmount from EmpProduct")

In [60]:
EmpProduct_DF.show()

+-----------+---------+----------------+-----------+
|EmployeeKey|FirstName|TotalProductCost|SalesAmount|
+-----------+---------+----------------+-----------+
|        282|    Linda|        973.4132|    939.588|
|        282|    Linda|         27.7564|     40.373|
|        287|      Shu|       5694.2832|   6074.982|
|        281|  Michael|         26.9598|     36.432|
|        283|  Jillian|       3755.9439|   4130.982|
|        283|  Jillian|          179.52|        288|
|        281|  Michael|        748.6284|    809.328|
|        281|  Michael|        2992.177|    3239.97|
|        295|   Rachel|         78.5178|    125.964|
|        295|   Rachel|         45.7965|      73.47|
|        284|  Garrett|       3633.8952|   3601.575|
|        288|     Jos�|       1239.4389|  1258.3767|
|        291|      Jae|         17.9776|     24.294|
|        282|    Linda|       5998.7824|  3691.3354|
|        283|  Jillian|        605.6492|   600.2625|
|        281|  Michael|         31.3418|     4

In [73]:
from pyspark.sql import UDFRegistration
from pyspark.sql.types import FloatType
from pyspark.sql import SQLContext

sqlc = SQLContext(sc)

C:\ProgramData\Anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [75]:
udf1 = UDFRegistration(sqlc)

In [76]:
def func(x):
    return sum(x) 

In [78]:
udf1.register(name='new_fun', f=lambda x : x + 10, returnType=FloatType())

AttributeError: 'SQLContext' object has no attribute '_jsparkSession'

In [1]:
arr = [[11, [1,2,3], [4,5,6,7]],
      [22, [10,20,30], [40,50,60,70]],
      [33, [100,200,300], [400,500,600,700]]]

In [6]:
arr[1][2][2]

60

In [7]:
arr[0]

[1, [1, 2, 3], [4, 5, 6, 7]]

In [8]:
arr[0][1]

[1, 2, 3]

In [9]:
arr = [(512.0, ([12.0, u'ML Mountain FrameW  Silver  42', 199.3757, u'Silver', 364.09, 218.454], [20070701.0, 20070713.0, 20070708.0, 442.0, 291.0, 1.0, 6.0, 2.0, 218.454, 0.0, 0.0, 199.3757, 398.7514, 436.908, 10.9227])), (512.0, ([12.0, u'ML Mountain FrameW  Silver  42', 199.3757, u'Silver', 364.09, 218.454], [20080501.0, 20080513.0, 20080508.0, 236.0, 289.0, 1.0, 1.0, 5.0, 218.454, 0.0, 0.0, 199.3757, 996.8785, 1092.27, 27.3068])), (512.0, ([12.0, u'ML Mountain FrameW  Silver  42', 199.3757, u'Silver', 364.09, 218.454], [20080201.0, 20080213.0, 20080208.0, 100.0, 291.0, 1.0, 6.0, 1.0, 218.454, 0.0, 0.0, 199.3757, 199.3757, 218.454, 5.4614])), (512.0, ([12.0, u'ML Mountain FrameW  Silver  42', 199.3757, u'Silver', 364.09, 218.454], [20071001.0, 20071013.0, 20071008.0, 437.0, 287.0, 1.0, 4.0, 1.0, 218.454, 0.0, 0.0, 199.3757, 199.3757, 218.454, 5.4614])), (512.0, ([12.0, u'ML Mountain FrameW  Silver  42', 199.3757, u'Silver', 364.09, 218.454], [20080401.0, 20080413.0, 20080408.0, 676.0, 285.0, 1.0, 5.0, 2.0, 218.454, 0.0, 0.0, 199.3757, 398.7514, 436.908, 10.9227]))]

In [30]:
arr[0][1]

([12.0, 'ML Mountain FrameW  Silver  42', 199.3757, 'Silver', 364.09, 218.454],
 [20070701.0,
  20070713.0,
  20070708.0,
  442.0,
  291.0,
  1.0,
  6.0,
  2.0,
  218.454,
  0.0,
  0.0,
  199.3757,
  398.7514,
  436.908,
  10.9227])

In [1]:
arr = [1,2,3,4,5]
a = [x*x for x in arr]
print(a)

[1, 4, 9, 16, 25]


In [2]:
pwd

'C:\\Users\\aditya.singh490'

In [3]:
mkdir adi

In [4]:
ls pwd

 Volume in drive C is OSDisk
 Volume Serial Number is 847B-7706

 Directory of C:\Users\aditya.singh490



File Not Found


In [8]:
f = open("C:\Users\aditya.singh490\Desktop\Steps.txt\", "r")

SyntaxError: EOL while scanning string literal (<ipython-input-8-f56565b8a571>, line 1)

In [9]:
cd C:\Users\aditya.singh490\Desktop\

C:\Users\aditya.singh490\Desktop


In [10]:
pwd

'C:\\Users\\aditya.singh490\\Desktop'

In [11]:
ls

 Volume in drive C is OSDisk
 Volume Serial Number is 847B-7706

 Directory of C:\Users\aditya.singh490\Desktop

09/28/2022  04:58 PM    <DIR>          .
09/28/2022  04:58 PM    <DIR>          ..
01/22/2022  12:28 AM           222,630 1.0_Intro_to_DBMS.pdf
01/22/2022  12:28 AM           348,712 1.1_Intro_to_DBMS.pdf
07/10/2022  04:21 PM           227,747 3.3_SQL_BasicQueries.pdf
07/10/2022  04:21 PM           227,747 3.3_SQL_BasicQueries-1.pdf
07/10/2022  04:21 PM         1,237,679 3.4_SQL_NestedQueries.pdf
08/28/2022  10:51 AM                65 Access Code.txt
07/18/2022  09:42 AM         3,540,873 app_annie__state_of_mobile_2021__en.pdf
08/18/2022  11:52 AM    <DIR>          Big Data Certifications
07/14/2022  12:28 AM         3,960,440 bigdatacert.pdf
11/21/2021  02:42 PM    <DIR>          Blockchain
05/24/2022  01:43 AM            37,174 Code.docx
09/27/2022  05:06 PM             1,086 Code.txt
08/27/2022  01:50 AM        51,695,709 Database using Microsoft SQL Server .pdf
07/11/20

In [14]:
f = open("Steps.txt", "w")

In [13]:
print(f.read())

1. Go to this link - https://gsi.learnondemand.net/
2. Login to Microsoft Account
3. Click on Redeem Training Key - 4D773C9FF8724108
4. View Agrrement -> Agree
5. First Launch Button
6. Password - Pa55w.rd

LET Portal - https://www.koenig-solutions.com/account/login

Attendance - https://rms-new.netlify.app/third_party?batch_header=125177

sillpipe - https://skillpipe.com/

QW7WJZ98H2DORI6OQI

nytfv8a - suffix for Azure resource


In [20]:
f.write("This file is opened in append mode")

34

In [22]:
f = open("Steps.txt", "r")

In [23]:
print(f.read())

This file is opened in write modeThis file is opened in append mode


In [27]:
def sqr(a):
    for i in range(len(a)):
        a[i] = a[i]*a[i]
    return a

arr = [1,2,3,4,5]
print(sqr(arr))

[1, 4, 9, 16, 25]


In [41]:
arr = [1,2,3,4,5]
arr = [x*x for x in arr]
print(a)

[1, 4, 9, 16, 25]


In [40]:
arr = [1,2,3,4,5]
arr = list(map(lambda x : x*x, arr))
print(arr)

[1, 4, 9, 16, 25]


In [42]:
arr = [1,2,'a',"bat",78]
a = [x for x in arr]
print(a)

[1, 2, 'a', 'bat', 78]
